# 3. RGSW Operation

Now we have all the required building blocks for RGSW operations.
We already checked RLWE' multiplication is working.

Call all previous methods

In [1]:
# Functions from previous lecturenote
import torch
import math

stddev = 3.2
logQ = 27

N = 2**10
Q = 2**logQ


def keygen(dim):
    return torch.randint(2, size = (dim,))

def errgen(stddev):
    e = torch.round(stddev*torch.randn(1))
    e = e.squeeze()
    return e.to(torch.int)

def errgen(stddev, N):
    e = torch.round(stddev*torch.randn(N))
    e = e.squeeze()
    return e.to(torch.int)

def uniform(dim, modulus):
    return torch.randint(modulus, size = (dim,))

def polymult(a, b, dim, modulus):
    res = torch.zeros(dim).to(torch.int)
    for i in range(dim):
        for j in range(dim):
            if i >= j:
                res[i] += a[j]*b[i-j]
                res[i] %= modulus
            else:
                res[i] -= a[j]*b[i-j] # Q - x mod Q = -x
                res[i] %= modulus

    res %= modulus
    return res

root_powers = torch.arange(N//2).to(torch.complex128)
root_powers = torch.exp((1j*math.pi/N)*root_powers)

root_powers_inv = torch.arange(0,-N//2,-1).to(torch.complex128)
root_powers_inv = torch.exp((1j*math.pi/N)*root_powers_inv)

def negacyclic_fft(a, N, Q):
    acomplex = a.to(torch.complex128)

    a_precomp = (acomplex[...,:N//2] + 1j * acomplex[..., N//2:]) * root_powers

    return torch.fft.fft(a_precomp)

def negacyclic_ifft(A, N, Q):
    b = torch.fft.ifft(A)
    b *= root_powers_inv

    a = torch.cat((b.real, b.imag), dim=-1)

    aint = a.to(torch.int32)
    # only when Q is a power-of-two
    aint &= Q-1

    return aint

In [2]:
# make an RLWE encryption of message
def encrypt_to_fft(m, sfft):
    ct = torch.stack([errgen(stddev, N), uniform(N, Q)])
    ctfft = negacyclic_fft(ct, N, Q)

    ctfft[0] += -ctfft[1]*sfft + negacyclic_fft(m, N, Q)

    return ctfft

def normalize(v, logQ):
    # same as follows but no branch
    """
    if v > Q//2:
        v -= Q
    """
    # vmod Q when Q is a power-of-two
    Q = (1 << logQ)
    v &= Q-1
    # get msb
    msb = (v & Q//2) >> (logQ - 1)
    v -= (Q) * msb
    return v

def decrypt_from_fft(ctfft, sfft):
    return normalize(negacyclic_ifft(ctfft[0] + ctfft[1]*sfft, N, Q), logQ)

In [3]:
# we will use B-ary decomposition, i.e., cut digits by logB bits
d = 3
logB = 7

decomp_shift = logQ - logB*torch.arange(d,0,-1).view(d,1)
mask = (1 << logB) - 1

gvector = 1 << decomp_shift

In [4]:
def decompose(a):
    
    assert len(a.size()) <= 2
    # for RLWE'
    if len(a.size()) == 1:
        res = (a.unsqueeze(0) >> decomp_shift.view(d, 1)) & mask
        return res
    # for RGSW
    elif len(a.size()) == 2:
        res = (a.unsqueeze(0) >> decomp_shift.view(d, 1, 1)) & mask
        return res

msbmask = 0
for i in decomp_shift:
    msbmask += (1<<(i+logB-1))

bin(msbmask)[2:]

# about twice heavier than unsigned decomposition
# it returns value -B/2 <= * <= B/2, not < B/2, but okay
def signed_decompose(a):
    # carry
    da = decompose(a + (a & msbmask))
    # -B
    da -= decompose((a & msbmask))
    return da

In [5]:
def encrypt_rlwep_fft(z, skfft):
    # it has a dimension of d, 2, N
    rlwep = torch.zeros(d, 2, N, dtype=torch.int32)

    # generate the 'a' part
    rlwep[:, 1, :] = torch.randint(Q, size = (d, N), dtype= torch.int32)

    # add error on b
    # INSECURE: need to be fixed later
    rlwep[:, 0, :] = torch.round(stddev * torch.randn(size = (d, N)))

    # following is equal to rlwep %= Q, but a faster version
    rlwep &= (Q-1)
    rlwep = normalize(rlwep, logQ)

    # do fft for easy a*s
    rlwepfft = negacyclic_fft(rlwep, N, Q)

    # now b = -a*sk + e
    rlwepfft[:, 0, :] -= rlwepfft[:, 1, :] * skfft.view(1,N//2)

    # add decomposition of z * vec{g}
    gz = gvector * z
    rlwepfft[:, 0, :] += negacyclic_fft(gz, N, Q)

    return rlwepfft

Test the RLWE' encryptions

1. Key gen and encryption

In [6]:
# generate two keys
s1 = keygen(N)
s2 = keygen(N)

s1fft = negacyclic_fft(s1, N, Q)
s2fft = negacyclic_fft(s2, N, Q)

In [7]:
m = torch.zeros((N), dtype=torch.int32)
m[2] = Q//4
m[3] = Q//4
m[5] = Q//4

m[:10]

tensor([       0,        0, 33554432, 33554432,        0, 33554432,        0,
               0,        0,        0], dtype=torch.int32)

In [8]:
ctfft = encrypt_to_fft(m, s1fft)
mdec = decrypt_from_fft(ctfft, s1fft)
mdec[:10]

tensor([       0,       -7, 33554428, 33554435,       -2, 33554431,        4,
              -1,       -3,        0], dtype=torch.int32)

2. Key switching key ($s_1 \rightarrow s_2$) gen

In [9]:
kskfft = encrypt_rlwep_fft(s1, s2fft)

In [10]:
kskfft.size()

torch.Size([3, 2, 512])

3. Try key switching

In [11]:
ct = negacyclic_ifft(ctfft, N, Q)

In [12]:
da = signed_decompose(ct[1])
dafft = negacyclic_fft(da, N, Q)

Following is
$a \odot RLWE'_{s2}(s1) = RLWE_{s2}(a \cdot s1)$

In [13]:
prodfft = dafft.view(d, 1, N//2) * kskfft
prodsumfft = torch.sum(prodfft, dim = 0)
prodsumfft[0] += ctfft[0]

Check if the decryption is successful.

In [14]:
mks = decrypt_from_fft(prodsumfft, s2fft)
print("m:\n",m[:10])
print("decrypted:\n",mks[:10])
print("decrypted (scaled to 1):\n",mks[:10]/(Q//4))

m:
 tensor([       0,        0, 33554432, 33554432,        0, 33554432,        0,
               0,        0,        0], dtype=torch.int32)
decrypted:
 tensor([   16803,    21622, 33572603, 33579753,    21249, 33564616,    23656,
           12367,     9222,    26401], dtype=torch.int32)
decrypted (scaled to 1):
 tensor([5.0077e-04, 6.4439e-04, 1.0005e+00, 1.0008e+00, 6.3327e-04, 1.0003e+00,
        7.0500e-04, 3.6857e-04, 2.7484e-04, 7.8681e-04])


Bonus: unsigned decomposition version

In [15]:
uda = decompose(ct[1])
udafft = negacyclic_fft(uda, N, Q)

In [16]:
uprodfft = udafft.view(d, 1, N//2) * kskfft
uprodsumfft = torch.sum(uprodfft, dim = 0)
uprodsumfft[0] += ctfft[0]

In [17]:
umks = decrypt_from_fft(uprodsumfft, s2fft)
print("m:\n",m[:10])
print("decrypted:\n",umks[:10])
print("decrypted (scaled to 1):\n",umks[:10]/(Q//4))



m:
 tensor([       0,        0, 33554432, 33554432,        0, 33554432,        0,
               0,        0,        0], dtype=torch.int32)
decrypted:
 tensor([   28635,    26631, 33583055, 33566950,    20196, 33587188,    15886,
           27219,    12966,    16934], dtype=torch.int32)
decrypted (scaled to 1):
 tensor([8.5339e-04, 7.9367e-04, 1.0009e+00, 1.0004e+00, 6.0189e-04, 1.0010e+00,
        4.7344e-04, 8.1119e-04, 3.8642e-04, 5.0467e-04])


In [18]:
print("signed")
print(torch.max(da), torch.min(da))
print("Error variance: \n", torch.var((m-mks)/Q))

print("unsigned")
print(torch.max(uda), torch.min(uda))
print("Error variance: \n", torch.var((m-umks)/Q))


signed
tensor(64) tensor(-64)
Error variance: 
 tensor(8.1394e-09)
unsigned
tensor(127) tensor(0)
Error variance: 
 tensor(1.7139e-08)
